## 0.前言

在训练网络时，一般的流程是通过前向计算获得网络的输出值，再通过损失函数计算网络误差，然后通过自动求导工具计算梯度并更新，同时间隔性地测试网络的性能。对于这种常用的训练逻辑，可以直接通过Keras提供的模型装配与训练等高层接口实现，简洁清晰。

Keras中两个特殊的类：

* `keras.layers.Layer`，网络层的母类，定义了网络层的常见功能如添加权值、管理权值列表等
* `keras.Model`，网络的母类，除了具有Layer类的功能，还具有保存模型、加载模型、训练与测试模型等便捷功能。`Sequential`也是`Model`的子类，具有`Model`的所有功能。

## 1.模型装配例子

以`Sequential`容器封装的网络为例，首先创建5层的全连接网络，用于MNIST手写数字图片识别，如下：

In [2]:
# 创建5层的全连接网络
from tensorflow import keras
from tensorflow.keras import layers
network=keras.Sequential([
    layers.Dense(256,activation='relu'),
    layers.Dense(128,activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(32,activation='relu'),
    layers.Dense(10)
])
network.build(input_shape=(4,28*28))
network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  200960    
_________________________________________________________________
dense_1 (Dense)              multiple                  32896     
_________________________________________________________________
dense_2 (Dense)              multiple                  8256      
_________________________________________________________________
dense_3 (Dense)              multiple                  2080      
_________________________________________________________________
dense_4 (Dense)              multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


创建网络后，常见流程：

1. 循环迭代多个Epoch
2. 按批产生训练数据
3. 前向计算
4. 计算误差
5. 反向传播
6. 更新参数

该流程很常用，因此被Keras通过`compile()`和`fit()`函数来实现。

### 1.1.装配

装配指通过`compile`函数指定网络的优化器、损失函数类型、评价指标等设定：

In [3]:
from tensorflow.keras import optimizers,losses

# 采用Adam优化器，学习率为0.01；采用交叉熵损失函数，包含Softmax
network.compile(optimizer=optimizers.Adam(lr=0.01),
                loss=losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'] # 设置测量指标为准确率
                )

### 1.2.训练

在模型装配后，通过`fit()`函数来输入数据集以训练：

In [4]:
# 加载数据集
import tensorflow as tf
from tensorflow.keras import datasets

def preprocess(x,y):
    # 调用此函数会自动传入x，y
    # 标准化到0~1
    x=tf.cast(x,dtype=tf.float32)/255.
    x=tf.reshape(x,[-1,28*28]) # 打平
    y=tf.cast(y,dtype=tf.int32) # 转换成整型张量
    y=tf.one_hot(y,depth=10) # 进行one-hot编码
    return x,y

def load_data():
    # 加载MNIST
    (x,y),(x_val,y_val)=datasets.mnist.load_data()
    batchsz=512
    # 构建数据集对象
    train_dataset=tf.data.Dataset.from_tensor_slices((x,y))
    train_dataset=train_dataset.shuffle(1000)
    #批量训练
    train_dataset=train_dataset.batch(batchsz)
    train_dataset=train_dataset.map(preprocess)
    train_dataset=train_dataset.repeat(20)

    # 加载验证/测试集
    val_dataset=tf.data.Dataset.from_tensor_slices((x_val,y_val))
    val_dataset=val_dataset.shuffle(1000).batch(batchsz).map(preprocess)
    return train_dataset,val_dataset


train_db,val_db=load_data()


# train_db为训练集，val_db为验证集，训练5个epochs，每两个epochs验证一次
# 将训练的信息保存到history对象中
history=network.fit(train_db,epochs=5,validation_data=val_db,validation_freq=2)

# print(history) # <tensorflow.python.keras.callbacks.History object at 0x7fb8be5e73d0>
print(history.history)

Epoch 1/5
2360/2360 [==============================] - 14s 6ms/step - loss: 0.0621 - accuracy: 0.9810

## 2.测试

通过`Model.predict(x)`完成模型的预测：

In [5]:
x,y=next(iter(val_db))
print(f'predict x: {x.shape}') # 打印当前batch的形状
out=network.predict(x) # 模型预测，预测结果在out中
print(out)

predict x: (512, 784)
[[  3.6948113  -14.412797    -3.4394515  ...  -0.23701444   8.919299
    4.7477446 ]
 [-57.19034     -6.7830315    9.024042   ...  45.038986    -9.376124
   15.0613575 ]
 [ 12.458674    23.24678     57.622505   ...  16.852085    14.916021
  -47.975548  ]
 ...
 [-29.315632   -74.886215   -91.17863    ... -22.586248    -0.37832558
   17.740868  ]
 [  6.950408   -21.60877    -14.106162   ...   1.7552286   22.518484
    7.8132606 ]
 [  8.026344    50.310215    15.786021   ...  11.270327     9.091065
   20.643475  ]]


循环测试完db数据集上所有样本，并打印出性能指标，例如：

In [6]:
network.evaluate(val_db)

20/20 [==============================] - 0s 3ms/step - loss: 0.1939 - accuracy: 0.9800


[0.1938883364200592, 0.9800000190734863]

In [ ]:
import os
pid=os.getpid()
!kill -9 $pid